# 1. 모듈 Import 및 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
%matplotlib inline

from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.metrics import mean_squared_error

from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
train_data = pd.read_csv('../data/최종데이터/for_train_data.csv')
test_data = pd.read_csv('../data/최종데이터/for_test_data.csv')
submission = pd.read_csv('./data/최종데이터/first_group_submission.csv', encoding='cp949')

In [ ]:
train_data.head(3)

,term,emd_nm,em_g_gap,em_g_gap_rate,em_g_avg,em_g_std,resid_reg_pop_gap,resid_reg_pop_gap_rate,resid_reg_pop_avg,resid_reg_pop_std,...,deli_per_use_amt_std,food_per_use_amt_gap,food_per_use_amt_gap_rate,food_per_use_amt_avg,food_per_use_amt_std,temperature,rain,fine_dust,em_g,peak
0,2018-06-01,건입동,3468900,1.016998,45007516,1486514.83,31.0,0.998265,9799.000000,12.68,...,395.39,10855.341526,1.069767,53964.320945,4530.37,21.721667,8.994167,31.633333,48187900,0
1,2018-06-01,남원읍,5900300,1.012975,40838033,2661150.77,66.0,1.003453,19144.000000,26.98,...,367.26,7150.172638,1.044615,34163.932414,3094.34,21.721667,8.994167,31.633333,42287450,0
2,2018-06-01,노형동,24880750,1.013611,259129650,10977885.73,52.0,0.999026,53351.666667,24.51,...,258.08,7942.755643,0.963592,39179.948301,3467.90,21.721667,8.994167,31.633333,272193365,0


In [ ]:
test_data.head(3)

,term,emd_nm,em_g_gap,em_g_gap_rate,em_g_avg,em_g_std,resid_reg_pop_gap,resid_reg_pop_gap_rate,resid_reg_pop_avg,resid_reg_pop_std,...,deli_per_use_amt_avg,deli_per_use_amt_std,food_per_use_amt_gap,food_per_use_amt_gap_rate,food_per_use_amt_avg,food_per_use_amt_std,temperature,rain,fine_dust,peak
0,2021-07-01,건입동,5989000,1.203722,32984150,2584274.49,67.0,0.993099,8941.000000,30.47,...,23822.779936,266.01,2663.530528,0.950798,43424.033709,1164.63,26.918548,6.504839,20.053763,8
1,2021-07-01,구좌읍,6452450,1.241856,30243200,2677238.86,118.0,0.993239,15455.666667,52.83,...,22662.817683,254.59,4771.683809,0.882261,32689.677020,2127.68,26.918548,6.504839,20.053763,8
2,2021-07-01,남원읍,6423350,1.102866,60246216,2888106.38,47.0,0.998827,18728.000000,19.20,...,21800.036803,138.04,3201.393896,0.946122,36743.632597,1320.03,26.918548,6.504839,20.053763,8


# 알수없음 / 19년부터 시행된 4개지역(구좌읍/조천읍/한경면/한림읍) 제외

In [ ]:
train_data.drop(train_data.loc[train_data['emd_nm']=='알수없음'].index, inplace=True)
train_data.drop(train_data.loc[train_data['emd_nm']=='구좌읍'].index, inplace=True)
train_data.drop(train_data.loc[train_data['emd_nm']=='조천읍'].index, inplace=True)
train_data.drop(train_data.loc[train_data['emd_nm']=='한경면'].index, inplace=True)
train_data.drop(train_data.loc[train_data['emd_nm']=='한림읍'].index, inplace=True)

In [ ]:
test_data.drop(test_data.loc[test_data['emd_nm']=='알수없음'].index, inplace=True)
test_data.drop(test_data.loc[test_data['emd_nm']=='구좌읍'].index, inplace=True)
test_data.drop(test_data.loc[test_data['emd_nm']=='조천읍'].index, inplace=True)
test_data.drop(test_data.loc[test_data['emd_nm']=='한경면'].index, inplace=True)
test_data.drop(test_data.loc[test_data['emd_nm']=='한림읍'].index, inplace=True)

# gap_rate, std가 이름에 포함된 변수삭제

In [ ]:
col_list = list(train_data.columns)
col_list

['term',
 'emd_nm',
 'em_g_gap',
 'em_g_gap_rate',
 'em_g_avg',
 'em_g_std',
 'resid_reg_pop_gap',
 'resid_reg_pop_gap_rate',
 'resid_reg_pop_avg',
 'resid_reg_pop_std',
 'foreign_pop_gap_rate',
 'foreign_pop_std',
 'total_pop_gap',
 'total_pop_gap_rate',
 'total_pop_avg',
 'total_pop_std',
 'f_resid_reg_rate_gap',
 'f_resid_reg_rate_gap_rate',
 'f_resid_reg_rate_avg',
 'f_resid_reg_rate_std',
 'f_foreign_rate_gap',
 'f_foreign_rate_gap_rate',
 'f_foreign_rate_avg',
 'f_foreign_rate_std',
 'f_total_rate_gap',
 'f_total_rate_gap_rate',
 'f_total_rate_avg',
 'f_total_rate_std',
 'floating_resd_rate_gap',
 'floating_resd_rate_gap_rate',
 'floating_resd_rate_avg',
 'floating_resd_rate_std',
 'floating_pop_cnt_gap',
 'floating_pop_cnt_gap_rate',
 'floating_pop_cnt_avg',
 'floating_pop_cnt_std',
 'floating_visit_rate_1020_gap',
 'floating_visit_rate_1020_gap_rate',
 'floating_visit_rate_1020_avg',
 'floating_visit_rate_1020_std',
 'floating_resd_rate_1020_gap',
 'floating_resd_rate_1020_gap_

In [ ]:
gap_rate = []
for variable in col_list:
    if variable.find('gap_rate') > 0:
        gap_rate.append(variable)
    else:
        pass
    
train_data.drop(['em_g_gap_rate', 'resid_reg_pop_gap_rate', 'foreign_pop_gap_rate', 'total_pop_gap_rate', 'f_resid_reg_rate_gap_rate',\
 'f_foreign_rate_gap_rate', 'f_total_rate_gap_rate', 'floating_resd_rate_gap_rate', 'floating_pop_cnt_gap_rate',\
 'floating_visit_rate_1020_gap_rate', 'floating_resd_rate_1020_gap_rate', 'floating_visit_rate_60up_gap_rate',\
'floating_resd_rate_60up_gap_rate', 'floating_f_visit_rate_gap_rate', 'floating_f_resd_rate_gap_rate', 'visit_pop_cnt_gap_rate_sf',\
 'rewo_pop_cnt_CVU_lf_gap_rate', 'rewo_pop_cnt_ETC_lf_gap_rate', 'visit_pop_cnt_CVU_lf_gap_rate', 'visit_pop_cnt_ETC_lf_gap_rate',\
 'floating_pop_cnt_CVU_lf_gap_rate', 'floating_pop_cnt_ETC_lf_gap_rate', 'floating_pop_cnt_lf_gap_rate', 'visit_rate_CVU_lf_gap_rate',\
 'visit_rate_ETC_lf_gap_rate', 'agri_use_amt_gap_rate', 'mart_use_amt_gap_rate', 'deli_use_amt_gap_rate', 'food_use_amt_gap_rate',\
 'agri_per_use_amt_gap_rate', 'mart_per_use_amt_gap_rate', 'deli_per_use_amt_gap_rate', 'food_per_use_amt_gap_rate'], axis=1, inplace=True)

test_data.drop(['em_g_gap_rate', 'resid_reg_pop_gap_rate', 'foreign_pop_gap_rate', 'total_pop_gap_rate', 'f_resid_reg_rate_gap_rate',\
 'f_foreign_rate_gap_rate', 'f_total_rate_gap_rate', 'floating_resd_rate_gap_rate', 'floating_pop_cnt_gap_rate',\
 'floating_visit_rate_1020_gap_rate', 'floating_resd_rate_1020_gap_rate', 'floating_visit_rate_60up_gap_rate',\
'floating_resd_rate_60up_gap_rate', 'floating_f_visit_rate_gap_rate', 'floating_f_resd_rate_gap_rate', 'visit_pop_cnt_gap_rate_sf',\
 'rewo_pop_cnt_CVU_lf_gap_rate', 'rewo_pop_cnt_ETC_lf_gap_rate', 'visit_pop_cnt_CVU_lf_gap_rate', 'visit_pop_cnt_ETC_lf_gap_rate',\
 'floating_pop_cnt_CVU_lf_gap_rate', 'floating_pop_cnt_ETC_lf_gap_rate', 'floating_pop_cnt_lf_gap_rate', 'visit_rate_CVU_lf_gap_rate',\
 'visit_rate_ETC_lf_gap_rate', 'agri_use_amt_gap_rate', 'mart_use_amt_gap_rate', 'deli_use_amt_gap_rate', 'food_use_amt_gap_rate',\
 'agri_per_use_amt_gap_rate', 'mart_per_use_amt_gap_rate', 'deli_per_use_amt_gap_rate', 'food_per_use_amt_gap_rate'], axis=1, inplace=True)

In [ ]:
std = []
for variable in col_list:
    if variable.find('std') > 0:
        std.append(variable)
    else:
        pass

train_data.drop(['em_g_std', 'resid_reg_pop_std', 'foreign_pop_std', 'total_pop_std', 'f_resid_reg_rate_std', 'f_foreign_rate_std',\
 'f_total_rate_std', 'floating_resd_rate_std', 'floating_pop_cnt_std', 'floating_visit_rate_1020_std',\
 'floating_resd_rate_1020_std', 'floating_visit_rate_60up_std', 'floating_resd_rate_60up_std', 'floating_f_visit_rate_std',\
 'floating_f_resd_rate_std', 'visit_pop_cnt_std_sf', 'rewo_pop_cnt_CVU_lf_std',\
 'rewo_pop_cnt_ETC_lf_std', 'visit_pop_cnt_CVU_lf_std', 'visit_pop_cnt_ETC_lf_std',\
 'floating_pop_cnt_CVU_lf_std', 'floating_pop_cnt_ETC_lf_std', 'floating_pop_cnt_lf_std', 'visit_rate_CVU_lf_std', \
'visit_rate_ETC_lf_std', 'agri_use_amt_std', 'mart_use_amt_std', 'deli_use_amt_std', 'food_use_amt_std',\
 'agri_per_use_amt_std', 'mart_per_use_amt_std', 'deli_per_use_amt_std', 'food_per_use_amt_std'], axis=1, inplace=True)

test_data.drop(['em_g_std', 'resid_reg_pop_std', 'foreign_pop_std', 'total_pop_std', 'f_resid_reg_rate_std', 'f_foreign_rate_std',\
 'f_total_rate_std', 'floating_resd_rate_std', 'floating_pop_cnt_std', 'floating_visit_rate_1020_std',\
 'floating_resd_rate_1020_std', 'floating_visit_rate_60up_std', 'floating_resd_rate_60up_std', 'floating_f_visit_rate_std',\
 'floating_f_resd_rate_std', 'visit_pop_cnt_std_sf', 'rewo_pop_cnt_CVU_lf_std',\
 'rewo_pop_cnt_ETC_lf_std', 'visit_pop_cnt_CVU_lf_std', 'visit_pop_cnt_ETC_lf_std',\
 'floating_pop_cnt_CVU_lf_std', 'floating_pop_cnt_ETC_lf_std', 'floating_pop_cnt_lf_std', 'visit_rate_CVU_lf_std', \
'visit_rate_ETC_lf_std', 'agri_use_amt_std', 'mart_use_amt_std', 'deli_use_amt_std', 'food_use_amt_std',\
 'agri_per_use_amt_std', 'mart_per_use_amt_std', 'deli_per_use_amt_std', 'food_per_use_amt_std'], axis=1, inplace=True)

# 추가 변수 제거 

## 4.1 그레인저 인과관계 확인후 인과관계 낮은 변수 제거 

In [ ]:
train_data.drop(['f_resid_reg_rate_avg',  'f_foreign_rate_gap', 'deli_per_use_amt_avg','floating_visit_rate_1020_gap', \
            'floating_f_resd_rate_gap', 'floating_visit_rate_60up_avg', 'visit_pop_cnt_gap_sf', 'agri_per_use_amt_gap'], 
           axis=1, inplace=True)

In [ ]:
test_data.drop(['f_resid_reg_rate_avg',  'f_foreign_rate_gap', 'deli_per_use_amt_avg','floating_visit_rate_1020_gap', \
            'floating_f_resd_rate_gap', 'floating_visit_rate_60up_avg', 'visit_pop_cnt_gap_sf', 'agri_per_use_amt_gap'], 
           axis=1, inplace=True)

## 4.2 후진 제거법으로 아이디어를 얻어서 제거한 변수들 

- deli_per_use_amt_gap
- visit_rate_CVU_lf_gap
- visit_pop_cnt_CVU_lf_gap
- floating_pop_cnt_lf_gap
- floating_pop_cnt_ETC_lf_gap
- rewo_pop_cnt_ETC_lf_gap
- floating_resd_rate_60up_avg
- floating_visit_rate_60up_gap
- floating_resd_rate_1020_avg
- floating_visit_rate_1020_avg
- floating_resd_rate_avg
- floating_resd_rate_gap
- resid_reg_pop_avg
- visit_pop_cnt_ETC_lf_avg
- visit_rate_ETC_lf_gap

# 5. 스케일링 및 X_train, y_train, X_test, y_test 분리

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
X_num_train = train_data[[ 'em_g_gap', 'em_g_avg', 'resid_reg_pop_gap',
        'total_pop_gap', 'total_pop_avg',
       'f_resid_reg_rate_gap', 'f_foreign_rate_avg', 'f_total_rate_gap',
       'f_total_rate_avg', 'floating_pop_cnt_gap', 'floating_pop_cnt_avg','floating_resd_rate_1020_gap',
       'floating_f_visit_rate_gap', 'floating_f_visit_rate_avg',
       'floating_f_resd_rate_avg', 'visit_pop_cnt', 'visit_pop_cnt_avg_sf',
        'rewo_pop_cnt_CVU_lf_gap', 'rewo_pop_cnt_CVU_lf_avg',      
        'rewo_pop_cnt_ETC_lf_avg', 'visit_pop_cnt_CVU_lf_avg',
       'visit_pop_cnt_ETC_lf_gap',
        'floating_pop_cnt_CVU_lf_avg','floating_pop_cnt_ETC_lf_gap', 'floating_pop_cnt_ETC_lf_avg',
        'floating_pop_cnt_lf_avg', 'visit_rate_CVU_lf_avg',
        'visit_rate_ETC_lf_avg', 'agri_use_amt_gap','agri_use_amt_avg', 'mart_use_amt_gap', 'mart_use_amt_avg',
       'deli_use_amt_gap', 'deli_use_amt_avg',
       'food_use_amt_avg', 'agri_per_use_amt_avg', 'mart_per_use_amt_gap',
       'mart_per_use_amt_avg', 'food_per_use_amt_gap'
               ,
       'food_per_use_amt_avg', 'temperature', 'rain', 'fine_dust','peak']]

scaler = StandardScaler()
scaler.fit(X_num_train)
X_train = scaler.transform(X_num_train)
X_train = pd.DataFrame(X_train, index=X_num_train.index, columns=X_num_train.columns)

y_train = train_data['em_g']

In [ ]:
X_num_test = test_data[[ 'em_g_gap', 'em_g_avg', 'resid_reg_pop_gap',
        'total_pop_gap', 'total_pop_avg',
       'f_resid_reg_rate_gap', 'f_foreign_rate_avg', 'f_total_rate_gap',
       'f_total_rate_avg','floating_pop_cnt_gap', 'floating_pop_cnt_avg','floating_resd_rate_1020_gap',
       'floating_f_visit_rate_gap', 'floating_f_visit_rate_avg', 'floating_f_resd_rate_avg', 'visit_pop_cnt', 'visit_pop_cnt_avg_sf',
       'rewo_pop_cnt_CVU_lf_gap', 'rewo_pop_cnt_CVU_lf_avg',      
        'rewo_pop_cnt_ETC_lf_avg', 'visit_pop_cnt_CVU_lf_avg',
       'visit_pop_cnt_ETC_lf_gap',             
        'floating_pop_cnt_CVU_lf_avg','floating_pop_cnt_ETC_lf_gap', 'floating_pop_cnt_ETC_lf_avg',
        'floating_pop_cnt_lf_avg', 'visit_rate_CVU_lf_avg',
       'visit_rate_ETC_lf_avg', 'agri_use_amt_gap','agri_use_amt_avg', 'mart_use_amt_gap', 'mart_use_amt_avg',
       'deli_use_amt_gap', 'deli_use_amt_avg',
       'food_use_amt_avg', 'agri_per_use_amt_avg', 'mart_per_use_amt_gap',
       'mart_per_use_amt_avg', 'food_per_use_amt_gap',
       'food_per_use_amt_avg', 'temperature', 'rain', 'fine_dust','peak']]

scaler = StandardScaler()
scaler.fit(X_num_test)
X_test = scaler.transform(X_num_test)
X_test = pd.DataFrame(X_test, index=X_num_test.index, columns=X_num_test.columns)

C:\Users\gram_\anaconda3\envs\min\lib\site-packages\sklearn\utils\extmath.py:770: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\gram_\anaconda3\envs\min\lib\site-packages\sklearn\utils\extmath.py:711: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


# 6. 학습

## 6.1 LGBM

In [ ]:
from lightgbm import LGBMRegressor
model_LGBM = LGBMRegressor(n_estimators=149, learning_rate=0.019, max_depth=5, num_leaves=70)


model_LGBM.fit(X_train, y_train)

LGBMRegressor(learning_rate=0.019, max_depth=5, n_estimators=149, num_leaves=70)

## 6.2 XGB

In [ ]:
from xgboost import XGBRegressor

model_XGB = XGBRegressor(learning_rate=0.125, n_estimators=141, max_depth=2)

model_XGB.fit(X_train, y_train)

[20:06:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(learning_rate=0.125, max_depth=2, n_estimators=141)

## 6.3 Voting 앙상블 학습

In [ ]:
from sklearn.ensemble import VotingRegressor

voting_clf = VotingRegressor(estimators=[('lgbm1',model_LGBM), ('lgbm2',model_LGBM),  ('xgb',model_XGB),('lgbm',model_LGBM)])

voting_clf.fit(X_train, y_train)

pred = voting_clf.predict(X_test)

[22:02:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


# 2021년 7월 8월 음식물 쓰레기 배출량 예측 

In [ ]:
pred = pred.round(1)

In [ ]:
pred

array([3.07583908e+07, 6.28600230e+07, 2.47963029e+08, 4.15155690e+07,
       1.09834026e+08, 5.99857121e+07, 1.67020022e+07, 1.05126375e+08,
       1.45493354e+07, 5.04039816e+07, 2.11844548e+07, 9.78551901e+07,
       3.96110889e+07, 6.31795589e+07, 2.39659832e+07, 1.13793435e+08,
       3.98699342e+07, 9.70104221e+07, 1.97614082e+08, 3.86979483e+07,
       1.68234789e+07, 5.29508398e+07, 9.21952958e+07, 2.54713390e+07,
       5.78674810e+07, 3.68144754e+07, 2.23698897e+08, 1.69584937e+07,
       2.13338587e+07, 1.39252243e+08, 1.87017925e+07, 6.23776583e+07,
       3.95770439e+07, 2.54659720e+07, 3.62996247e+07, 1.02274471e+08,
       2.38951448e+07, 3.42669626e+07, 5.75923636e+07, 2.36247492e+08,
       4.14063482e+07, 1.07384530e+08, 5.44103637e+07, 1.68931999e+07,
       9.48695322e+07, 1.49288518e+07, 5.04977988e+07, 2.08629372e+07,
       9.44878681e+07, 3.87940655e+07, 5.81570448e+07, 2.57431376e+07,
       1.07088224e+08, 4.24003307e+07, 9.69745176e+07, 1.79792122e+08,
      

In [ ]:
submission['em_g_7'] = pred[:37]
submission['em_g_8'] = pred[37:]

In [ ]:
submission.to_csv('그룹1_7월8월_음식물쓰레기예측.csv', index=False)